In [113]:
# works good
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import sys
import sklearn
from sklearn.preprocessing import StandardScaler
import sklearn.preprocessing as preprocessing
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt  
np.set_printoptions(threshold=sys.maxsize)   
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 300)
pd.set_option('display.min_rows', 300)
pd.set_option('display.expand_frame_repr', True)

def change_dir(
        index):
    dir_win = 'J:/project files/lab_files/lump_project/data/moldir_blind_exp/'
    dir_mac = '/Volumes/Samsung_T5/project files/lab_files/lump_project/data/moldir_blind_exp/'
    if index == 1:
        directory = dir_mac
    else:
        directory = dir_win
    os.chdir(directory)
# dl_percent_without_glove = calc_percent_by_participant(df_without_glove)
# dl_percent_with_glove = calc_percent_by_participant(df_with_glove)
def dl_percent(
    df):
    N = 8 # number of trials per single participant palpating single prototype
    # calculate average of the each participant for each prototype
    # add the label for each participant and prototype
    # the structure of the ouput data is the following 
    # LP_p Participant Protoype
    # 0.8. 0.          0
    # 0.7. 1.          0
    # .
    # .
    # 0.7. 10.          9
    dl_percent = pd.DataFrame(df["LP_p"].groupby(df["LP_p"].index // N).sum()/N)
    dl_percent.reset_index(drop=True)
    dl_percent = pd.concat([dl_percent, \
                pd.DataFrame(df["Participant"].iloc[::8].reset_index(drop=True)),\
                pd.DataFrame(df["Prototype"].iloc[::8].reset_index(drop=True))],axis = 1)
    return dl_percent

def reshape_one_case(
    dl, column_labels):
    # dl -> data in the form -> Accuracy Participant Prototype
    #                           0.8      1           0
    #                           0.8      2           0
    #                           ...      ...         ...
    #                           0.8      10          0
    #                           ...      ...         ...
    #                           0.8      10          9
    # column_labels -> labels for each column representing the prototype
    # reahspe the data in the following form
    # rows -> participants 
    # columns prototypes NG_SH1.. NG_NL
    dl_transp = pd.DataFrame()
    temp = pd.DataFrame()
    for i in range(9):
        temp = dl.iloc[i*10:(i+1)*10, 0]
        temp = pd.DataFrame(temp.reset_index(drop = True))
        dl_transp = pd.concat([dl_transp, temp], axis = 1)
        dl_transp = pd.DataFrame(dl_transp.reset_index(drop = True))
    dl_transp.columns = column_labels
    return dl_transp

def reshape_R_one_condition(
    df, condition_index):
    # initialize the column names for each prototype 
    # for casse with glove and for casse without glove
    # NG - no glove
    # NL -> no lump
    if (condition_index == True):
        column_l = ['WG_' + label for label in prot_labels]
        column_no_l = ['WG_' + "NL"]
    else:
        column_l = ['NG_' + label for label in prot_labels]
        column_no_l = ['NG_' + "NL"] 
    dl_l_percent = pd.DataFrame()
    dl_no_l_percent = pd.DataFrame()
    # calculate the accuracy for each participant for each prototype
    df_l = df[0:len(df)//2]
    df_no_l = df[len(df)//2:].reset_index(drop = True)
    dl_l_percent = dl_percent(df_l)
    dl_no_l_percent = 1 - dl_percent(df_no_l)
    
    dl_l = reshape_one_case(dl_l_percent, column_l)
    dl_no_l = reshape_one_case(dl_no_l_percent, column_no_l*9)
    dl_no_l = pd.DataFrame(np.mean(dl_no_l, axis = 1))
    dl_no_l.columns = (column_no_l)
    
    return pd.concat([dl_l,dl_no_l], axis = 1)
# change directory and load data for two conditions
index = 2
change_dir(index)
os.listdir()
df_with_glove = pd.read_csv("df_with_glove.csv", sep=',')
df_without_glove = pd.read_csv("df_without_glove.csv", sep=',')
##########################################
df_NG = reshape_R_one_condition(df_without_glove, False)
df_WG = reshape_R_one_condition(df_with_glove, True)
df_R_studio = pd.concat([df_NG, df_WG], axis = 1)
df_R_studio.to_csv("df_R_studio.csv", index = False)